In [6]:
import pandas as pd
import numpy as np

In [7]:
import sys  
sys.path.insert(0, '../../scripts')

from D_plot_specificity_matrix_utils import calc_binding_concordance

In [8]:
def notnan(x):
    return x == x

In [9]:
def get_multiplets(df):
    #tmp = df[idx1 & idx2]
    dct = df.groupby(['ct','peptide_HLA']).gem.count() > 1
    idx = df.set_index(['ct','peptide_HLA']).index.map(dct)
    return idx.fillna(False)

# Input

In [1]:
#EXP = '../../experiments/exp10x/run1/cat/'
#EXP = '../../experiments/exp13/run3/cat/'
#EXP = '../../experiments/exp13/run2/cat/'
EXP = '../../experiments/exp09/run1/cat/'

In [2]:
VALID = EXP + 'eval_clonotypes/valid_ct.csv'
#VALID = EXP + 'tables/tcr_barcode.cleaned.valid_ct.csv' # exp13_2
THRSH = EXP + 'eval_clonotypes/threshold/opt.csv'

# Output

In [40]:
OUT = '../tmp_files/model_data/exp09.csv'

# Load

In [10]:
df = pd.read_csv(VALID)

In [11]:
opt_thr = pd.read_csv(THRSH, index_col=0, header=None, names=['thr']).thr.dropna()

In [12]:
opt_thr

umi_count_mhc    2.000000
delta_umi_mhc    1.636036
umi_count_TRA    1.000000
delta_umi_TRA    0.757858
umi_count_TRB    2.000000
delta_umi_TRB    0.757858
Name: thr, dtype: float64

In [13]:
df.fillna({'umi_count_mhc':0, 'delta_umi_mhc':0, 'umi_count_mhc_rel':0,
           'umi_count_cd8':0, 'delta_umi_cd8':0,
           'umi_count_TRA':0, 'delta_umi_TRA':0,
           'umi_count_TRB':0, 'delta_umi_TRB':0}, inplace=True) #'cdr3_TRA':'','cdr3_TRB':''

# Filters

In [15]:
idx1 = eval(' & '.join([f'(df.{k} >= {abs(v)})' for k,v in opt_thr.items()]))

#idx2 = df.hto_global_class == 'Singlet'

idx3 = df.apply(lambda row: row.peptide_HLA.split()[-1] in row.HLA_cd8 if (notnan(row.peptide_HLA) & notnan(row.HLA_cd8)) else False, axis=1)

idx4 = df['exclude_single-chain_TCRs']

idx5 = get_multiplets(df)

In [15]:
idx10x = ~(df.peptide_HLA.isin(['GILGFVFTL A0201','RAKFKQLL B0801','KLGGALQAK A0301']) & idx5)

# Main

In [19]:
sub_df = calc_binding_concordance(df[idx1].copy(), 'ct') # exp9
#sub_df = calc_binding_concordance(df[idx1 & idx3].copy(), 'ct')
#sub_df = calc_binding_concordance(df[idx1 & idx3 & idx10x].copy(), 'ct') # exp10x

In [20]:
sub_df.shape

(998, 105)

In [21]:
sub_df = sub_df[sub_df.binding_concordance > 0.5].copy()

In [22]:
sub_df.dropna(subset=['cdr3_TRA','cdr3_TRB'], inplace=True)
sub_df.drop_duplicates(subset=['peptide_HLA','cdr3_TRA','cdr3_TRB'], inplace=True)

In [23]:
sub_df.sort_values(by=['ct','pep_match','binding_concordance'], inplace=True)

In [24]:
sub_df.drop_duplicates(subset=['ct'], keep='last', inplace=True)

In [25]:
idx6 = (sub_df.cdr3_TRA.str.len() >= 8) & (sub_df.cdr3_TRA.str.len() <= 19)
idx7 = (sub_df.cdr3_TRB.str.len() >= 8) & (sub_df.cdr3_TRB.str.len() <= 19)

In [26]:
sub_df = sub_df[idx6 & idx7].copy()

In [27]:
sub_df['label'] = 1

In [28]:
sub_df.peptide_HLA.value_counts()

VTEHDTLLY A0101      48
RAKFKQLL B0801       22
TPRVTGGGAM B0702     12
NLVPMVATV A0201       3
EERQAYLTNQY A0101     3
FSAGAFILYK A0101      2
ALPGVPPV A0201        1
TNQYMQRMASV A2401     1
SVDIHFDPGF A0101      1
Name: peptide_HLA, dtype: int64

In [29]:
sub_df[['peptide_HLA','cdr3_TRA','cdr3_TRB', 'label']]

,peptide_HLA,cdr3_TRA,cdr3_TRB,label
14,TPRVTGGGAM B0702,CAASQNEKLTF,CASRIGAAGNSPLHF,1
0,RAKFKQLL B0801,CALRDMEYGNKLVF,CASSLIVSGGANEQFF,1
17,NLVPMVATV A0201,CAVNYGNKLVF,CASSLVAGAPSEQYF,1
19,VTEHDTLLY A0101,CAVTAPAYSGAGSYQLTF,CASSYTPGSTYEQYF,1
92,VTEHDTLLY A0101,CAAFRAYNTDKLIF,CSARGLGQAKEYGYTF,1
...,...,...,...,...
1117,VTEHDTLLY A0101,CAAFRAYNTDKLIF,CASSYRTVSSYEQYF,1
1164,VTEHDTLLY A0101,CAAFRAYNTDKLIF,CASSYTPGSTYEQYF,1
28,VTEHDTLLY A0101,CAVAAPAYSGAGSYQLTF,CASSYTPGSTYEQYF,1
1001,VTEHDTLLY A0101,CAVNYGNKLVF,CATRQNTEAFF,1


In [30]:
ct2pep = df.groupby('ct').peptide_HLA.unique().to_frame().rename(columns={'peptide_HLA':'pos_pep'}).reset_index()

In [31]:
all_pep = set(df.peptide_HLA.unique())
all_pep

{'AIFCAIQNL A0201',
 'ALPGVPPV A0201',
 'AMLIRDRL B0801',
 'ATTADVDAGSL A0101',
 'AYLTNQYM A2401',
 'CSSVDIHF A0101',
 'DEYVHNTTA B0801',
 'EERQAYLTNQY A0101',
 'FLDTNQVV A0201',
 'FLDTNQVVRI A0201',
 'FSAGAFILYK A0101',
 'FSAGAFILYN A0101',
 'GLFSAGAF A2401',
 'GPFPGNLV B0702',
 'GPFPGNLVD B0702',
 'GPFPGNLVDV B0702',
 'GPKPAKPCS B0702',
 'GPKPAKPCSGA B0702',
 'GPRSERRKW B0702',
 'HTGEKHYAC B0801',
 'ICSSVDIHF A0101',
 'IHFDPGFEEQ A0101',
 'IHFDPGFEEQL A2401',
 'ILKEPVHGV A0201',
 'ILLFERPHTM B0801',
 'IQFTSFVAV A0201',
 'IRHKRQSW B0801',
 'IRHKRQSWV B0801',
 'LFSAGAFILYN A0101',
 'LHPSTPLKPL B0702',
 'LLFERPHTM B0801',
 'MDMFSDEQLKV A2401',
 'NLVPMVATV A0201',
 'PSTPLKPL B0702',
 'QAYLTNQY A0101',
 'RAKFKQLL B0801',
 'RQAYLTNQY A0101',
 'RQSWVNATL B0702',
 'RRKVPAQRL B0801',
 'RTMFWGPEV A0201',
 'SAGAFILYK A0101',
 'SIKDGLPPV A0201',
 'SLIIQFTSFV A0201',
 'SQKVLRHM B0801',
 'SQSDQTSEL B0801',
 'SSGYRIGQF B0801',
 'SVDIHFDPG A0101',
 'SVDIHFDPGF A0101',
 'TADVDAGSLS A0101',
 'TNQYMQRM

In [32]:
ct2pep['peptide_HLA'] = ct2pep.apply(lambda row: list(all_pep - set(row.pos_pep)), axis=1)

In [33]:
ct2pep

,ct,pos_pep,peptide_HLA
0,1.0,"[VTEHDTLLY A0101, RAKFKQLL B0801, TPRVTGGGAM B...","[GPKPAKPCS B0702, LLFERPHTM B0801, GLFSAGAF A2..."
1,2.0,"[RAKFKQLL B0801, VTEHDTLLY A0101, EERQAYLTNQY ...","[LFSAGAFILYN A0101, LLFERPHTM B0801, FSAGAFILY..."
2,3.0,"[PSTPLKPL B0702, NLVPMVATV A0201, EERQAYLTNQY ...","[HTGEKHYAC B0801, LFSAGAFILYN A0101, GPKPAKPCS..."
3,4.0,"[VTEHDTLLY A0101, RAKFKQLL B0801, EERQAYLTNQY ...","[AMLIRDRL B0801, HTGEKHYAC B0801, GPKPAKPCS B0..."
4,5.0,"[VTEHDTLLY A0101, NLVPMVATV A0201, RAKFKQLL B0...","[AMLIRDRL B0801, HTGEKHYAC B0801, GPKPAKPCS B0..."
...,...,...,...
323,633.0,[SVDIHFDPG A0101],"[AMLIRDRL B0801, HTGEKHYAC B0801, GPKPAKPCS B0..."
324,634.0,[VTEHDTLLY A0101],"[AMLIRDRL B0801, HTGEKHYAC B0801, GPKPAKPCS B0..."
325,636.0,[FLDTNQVV A0201],"[AMLIRDRL B0801, HTGEKHYAC B0801, GPKPAKPCS B0..."
326,646.0,"[RAKFKQLL B0801, VTEHDTLLY A0101]","[AMLIRDRL B0801, HTGEKHYAC B0801, GPKPAKPCS B0..."


In [34]:
neg_df = pd.merge(sub_df[['ct','cdr3_TRA','cdr3_TRB']], ct2pep, on='ct', how='left').explode('peptide_HLA')

In [35]:
neg_df['label'] = 0

In [36]:
out_df = pd.concat([sub_df[['peptide_HLA','cdr3_TRA','cdr3_TRB', 'label']], neg_df[['peptide_HLA','cdr3_TRA','cdr3_TRB', 'label']]])

In [37]:
pMHC = out_df.peptide_HLA.str.split(' ', expand=True)
out_df['peptide'] = pMHC[0]
out_df['mhc'] = pMHC[1]

In [38]:
out_df

,peptide_HLA,cdr3_TRA,cdr3_TRB,label,peptide,mhc
14,TPRVTGGGAM B0702,CAASQNEKLTF,CASRIGAAGNSPLHF,1,TPRVTGGGAM,B0702
0,RAKFKQLL B0801,CALRDMEYGNKLVF,CASSLIVSGGANEQFF,1,RAKFKQLL,B0801
17,NLVPMVATV A0201,CAVNYGNKLVF,CASSLVAGAPSEQYF,1,NLVPMVATV,A0201
19,VTEHDTLLY A0101,CAVTAPAYSGAGSYQLTF,CASSYTPGSTYEQYF,1,VTEHDTLLY,A0101
92,VTEHDTLLY A0101,CAAFRAYNTDKLIF,CSARGLGQAKEYGYTF,1,VTEHDTLLY,A0101
...,...,...,...,...,...,...
92,CSSVDIHF A0101,CAVEDRLDNDMRF,CASSLIVSGGANEQFF,0,CSSVDIHF,A0101
92,YPLVKMRLSV B0702,CAVEDRLDNDMRF,CASSLIVSGGANEQFF,0,YPLVKMRLSV,B0702
92,EERQAYLTNQY A0101,CAVEDRLDNDMRF,CASSLIVSGGANEQFF,0,EERQAYLTNQY,A0101
92,ALPGVPPV A0201,CAVEDRLDNDMRF,CASSLIVSGGANEQFF,0,ALPGVPPV,A0201


In [39]:
out_df.groupby(['peptide_HLA','label']).size()

peptide_HLA        label
AIFCAIQNL A0201    0        92
ALPGVPPV A0201     0        89
                   1         1
AMLIRDRL B0801     0        89
ATTADVDAGSL A0101  0        88
                            ..
VTEHDTLLY A0101    1        48
WPRAPRGQ B0702     0        90
WPRAPRGQI B0801    0        92
YPLVKMRLS B0702    0        93
YPLVKMRLSV B0702   0        90
Length: 67, dtype: int64

In [41]:
out_df[['peptide','mhc','cdr3_TRA','cdr3_TRB','label']].to_csv(OUT, index=False, header=False)